In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## Scraping

In [51]:
def scrape_page(page_n):
    url = f"https://www.quandoo.fi/en/helsinki?districtFilter=3637&bookable=true&onlySpecialOffers=false&page={page_n}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rest_cards = soup.find_all(attrs={"data-qa": "merchant-card"})

    rest_names = [card.find('h3').text.strip() for card in rest_cards]
    rest_locations = [card.find(attrs={"data-qa": "merchant-location"}).text.strip() for card in rest_cards]
    rest_cuisines = [card.find(attrs={"data-qa": "merchant-card-cuisine"}).text.strip() for card in rest_cards]
    
    rest_meals_boxes = [card.find(attrs={"data-qa": "merchant-meal"}) for card in rest_cards]
    rest_meals = [box.text.strip() if box else None for box in rest_meals_boxes]
    
    rest_rating_boxes = [card.find(attrs={"data-qa": "reviews-score"}) for card in rest_cards]
    rest_ratings = [float(box.text.split('/')[0].strip()) if box else None for box in rest_rating_boxes]

    rest_review_boxes = [card.find(class_="sc-1atis9w-3 dfyExP") for card in rest_cards]
    rest_review_counts = [int(box.text.split()[0].strip()) if box else None for box in rest_review_boxes]

    rest_price_boxes = [card.find(class_=re.compile(r'.*price-indicator')) for card in rest_cards]
    rest_price_levels = [len(box.find_all(class_=re.compile(r'.*oGCHK'))) for box in rest_price_boxes]

    rest_page_urls = [card.find('a')['href'] for card in rest_cards]

    df = pd.DataFrame({'Name': rest_names, 
                    'Location': rest_locations,
                    'Cuisine': rest_cuisines,
                    'Meals': rest_meals,
                    'Price Level (out of 4)': rest_price_levels,
                    'Rating (out of 6)': rest_ratings,
                    'Review Count': rest_review_counts,
                    'Page URL': rest_page_urls
                    })

    return df

In [52]:
restaurant_data = pd.DataFrame()

for page_n in range(1, 15):
    page_data = scrape_page(page_n)
    restaurant_data = pd.concat([restaurant_data, page_data], ignore_index=True)

restaurant_data

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
0,Luovuus kukkii kaaoksesta,Located at Kaartinkaupunki area,International Restaurant,Meals: Dinner,3,5.8,469.0,/en/place/luovuus-kukkii-kaaoksesta-90397/menu
1,Ravintola MyStuu,Located at Punavuori area,Swiss Restaurant,"Meals: Lunch, Dessert, Dinner",3,5.6,292.0,/en/place/ravintola-my-stuu-98898/menu
2,Gaucho,Located at City Centre area,Brazilian Restaurant,Meals: Dinner,3,5.6,1111.0,/en/place/gaucho-105125/menu
3,Finlandia Caviar,Located at City Centre area,Gourmet Restaurant,"Meals: Lunch, Dinner",3,5.6,143.0,/en/place/finlandia-caviar-15896/menu
4,Restaurant Armenian House,Located at Kamppi area,International Restaurant,"Meals: Lunch, Dinner",2,5.6,314.0,/en/place/armenian-house-55148/menu
...,...,...,...,...,...,...,...,...
338,Lie Mi Lunch Kitchen,Located at Ruoholahti area,Vietnamese Restaurant,"Meals: Lunch, Brunch",2,NaN,NaN,/en/place/lie-mi-lunch-kitchen-106571/menu
339,Kahvila Mutteri,Located at Lauttasaari area,Dessert,Meals: Cake & Coffee,2,NaN,NaN,/en/place/kahvila-mutteri-100773/menu
340,Villa Alia,Located at Kulosaari area,Finnish Restaurant,"Meals: Lunch, Brunch, Sunday lunch",2,5.3,7.0,/en/place/villa-alia-102352/menu
341,Merisali - Hilton Kalastajatorppa,Located at Munkkiniemi area,Scandinavian Restaurant,"Meals: Buffet, Dinner",3,5.0,3.0,/en/place/merisali-hilton-kalastajatorppa-9200...


In [ ]:
restaurant_data.to_csv("restaurant_data.csv", sep="\t")

## Prettifying

In [56]:
data = pd.read_csv("restaurant_data.csv", sep="\t", index_col=0)
data.sample(10)

,Name,Location,Cuisine,Meals,Price Level (out of 4),Rating (out of 6),Review Count,Page URL
19,Rioni Helsinki,Located at Kaartinkaupunki area,Georgian Restaurant,"Meals: Lunch, Dinner",3,5.4,988.0,/en/place/rioni-helsinki-93704/menu
135,The Schnitzel Tripla,Located at Pasila area,Korean Restaurant,"Meals: Lunch, Dinner",2,4.3,63.0,/en/place/the-schnitzel-tripla-90435/menu
38,Lie Mi Kamppi,Located at Kamppi area,Vietnamese Restaurant,"Meals: Lunch, Dinner",2,5.3,71.0,/en/place/lie-mi-kamppi-105063/menu
329,Cafe & Bistro Hiezu,Located at Töölö area,European Restaurant,"Meals: Brunch, Dinner",2,4.5,45.0,/en/place/cafe-bistro-hiezu-96430/menu
237,Tran Tinh,Located at Hakaniemi area,Vegan Option,"Meals: Lunch, Dinner",2,NaN,NaN,/en/place/tran-tinh-restaurant-104989/menu
325,m/s Natalia – Royal Line,Located at Kruununhaka area,Scandinavian Restaurant,"Meals: Buffet, Lunch, Dinner",3,4.5,198.0,/en/place/ms-natalia-royal-line-88255/about
104,Da Spizzico,Located at Jätkäsaari area,Italian Restaurant,"Meals: Lunch, Dinner",3,5.4,42.0,/en/place/da-spizzico-100366/menu
73,Alice Italian,Located at Vallila area,Italian Restaurant,"Meals: Breakfast, Lunch, Dinner",2,5.0,198.0,/en/place/alice-italian-48550/about
182,Wine & Tapas Helsinki,Located at Kamppi area,Spanish Restaurant,"Meals: Tapas, Lunch, Dinner",2,5.3,3.0,/en/place/wine-tapas-helsinki-107955/menu
11,Pompier ESPA,Located at Kaartinkaupunki area,International Restaurant,"Meals: Lunch, Brunch, Dinner",3,5.5,1361.0,/en/place/pompier-espa-19553/menu
